# A Cando-app to setup free energy perturbation calculations using AMBER

First we do the following:

  * Load the TIRUN package.
  * Set things up to carry out geometry optimization using Amber.

In [ ]:
startSwank 4011

In [ ]:
(ql:quickload :tirun)

In [ ]:
(leap:setup-amber-paths)

In [ ]:
(source "leaprc.protein.ff14SB")

# Start a new TIRUN calculation

In [ ]:
(defparameter *tiruns* (make-instance 'tirun:calculation))

# Import the sketch from a Chemdraw file

In [ ]:
(jupyter:png-file "ligand2.png" )

In [ ]:
(defparameter *sketch* (tirun:load-chem-draw-tirun "ligand2.cdxml"))

In [ ]:
(defparameter *cdagg* (chem:as-aggregate *sketch*))

-------------

In [ ]:
as = (tirun::build-assembly *sketch* :verbose t)

In [ ]:
allligands = (tirun:setup-ligands *tiruns* *sketch*)

In [ ]:
(defparameter v (show (tirun:layout-ligands *tiruns*)))
v

In [ ]:
(tirun:layout-ligands *tiruns*)

# Load the Thrombin PDB file

In [ ]:
*default-pathname-defaults*

In [ ]:
(defparameter *thrombin* (load-pdb "2zff_fixed.pdb" :ignore-missing-topology t))

In [ ]:
(show *thrombin*)

In [ ]:
(build-unbuilt-hydrogens *thrombin*)

In [ ]:
(defparameter vthrombin (show *thrombin*))
;;;(nglv::add-representation vthrombin "surface" :selection "protein")
vthrombin

In [ ]:
(nglv::add-representation vthrombin "ball+stick" :selection "all")

In [ ]:
(remove-molecules *thrombin* 'solvent)

In [ ]:
(cando:do-molecules (m *thrombin*) (format t "mol: ~a number-of-atoms -> ~a~%" m (chem:number-of-atoms m))
                    (cando:do-residues (res m) (format t "  res -> ~a~%" res)))

In [ ]:
(chem:setf-molecule-type (chem:content-with-name *thrombin* :|53U_3|) 'ligand)

In [ ]:
ligmol = (chem:content-with-name *thrombin* :|53U_3|)

In [ ]:
(chem:remove-molecule *thrombin* ligmol)

In [ ]:
show *thrombin*

In [ ]:
show ligmol

In [ ]:
(pushnew *thrombin* (tirun:receptors *tiruns*))

# Load a template ligand

# Use the GAFF force field to build chemically reasonable structures of candidate ligands

In [ ]:
(clear-force-field)
(load-atom-type-rules "ATOMTYPE_GFF.DEF")
(load-amber-params "gaff.dat")

In [ ]:
(show (tirun:layout-ligands *tiruns* :accessor 'tirun::molecule))

In [ ]:
mol = (tirun::molecule (first (tirun::ligands *tiruns*)))

In [ ]:
show mol

In [ ]:
(chem:map-bonds nil (lambda (a b o) (format t "~a -> ~a ~a~%" a b o)) mol)

# Pose the new ligands onto the template ligand

In [ ]:
(tirun::pose-ligands-using-similarity *tiruns* ligmol)

In [ ]:
(defparameter vthrombin (show *thrombin*))
;;;(nglv::add-representation vthrombin "surface" :selection "protein")
vthrombin

In [ ]:
(defparameter *lig* (chem:make-aggregate))
(chem:add-matter *lig* ligmol)
(nglv::add-structure vthrombin (make-instance 'cando-structure :matter *lig*))
(defparameter *moveable-agg* (chem:make-aggregate))
(chem:add-matter *moveable-agg* (tirun::molecule (third (tirun:ligands *tiruns*))))
(nglv::add-structure vthrombin (make-instance 'cando-structure :matter *moveable-agg*))

In [ ]:
(show (tirun:layout-ligands *tiruns* :accessor 'tirun::molecule))

# Define the pairs of compounds between which we want to carry out free energy perturbation calculations

In [ ]:
(tirun:build-job-nodes *tiruns*)

In [ ]:
(tirun::connect-job-nodes *tiruns* :simple :connections 2 :stages 3 :windows 11)

In [ ]:
(tirun:jobs *tiruns*)

In [ ]:
(graph::save-graph *tiruns*)

In [ ]:
*default-pathname-defaults*

In [ ]:
(defparameter *work-list* (tirun::generate-jobs *tiruns*))

In [ ]:
*work-list*